In [4]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import warnings
import itertools
import matplotlib.pyplot as plt
import statsmodels.api as sm


# SARIMA

In [ ]:
# Load the data
data = pd.read_csv('USA17S.csv', engine='python', skipfooter=3)
data['date']=pd.to_datetime(data['date'], format='%Y-%m-%d')
data.set_index(['date'], inplace=True)

# Plot the data
data.plot()
plt.ylabel('Monthly airline passengers (x1000)')
plt.xlabel('date')
plt.show()

plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')

In [ ]:
# Define the d and q parameters to take any value between 0 and 1
q = d = range(0, 4)
# Define the p parameters to take any value between 0 and 3
p = range(0, 4)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
# Define the d and q parameters to take any value between 0 and 1
q = d = range(0, 4)
# Define the p parameters to take any value between 0 and 3
p = range(0, 4)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
train_data = data['2006-01-31':'2016-12-31']
test_data = data['2017-01-31':'2017-12-31']
warnings.filterwarnings("ignore") # specify to ignore warning messages

AIC = []
SARIMAX_model = []
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMAX{}x{} - AIC:{}'.format(param, param_seasonal, results.aic), end='\r')
            AIC.append(results.aic)
            SARIMAX_model.append([param, param_seasonal])
        except:
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(train_data,
                                order=SARIMAX_model[AIC.index(min(AIC))][0],
                                seasonal_order=SARIMAX_model[AIC.index(min(AIC))][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

In [ ]:
pred2 = results.get_forecast('2018-01-31')
pred2_ci = pred2.conf_int()
print(pred2.predicted_mean['2017-01-31':'2017-12-31'])
print(pred2_ci)
tt=pred2.predicted_mean['2017-01-31':'2017-12-31'].values

In [ ]:
np.savetxt('SARIMA.csv', tt, delimiter = ',')
dat = pd.read_csv('USA17.csv')
OO = np.array(dat["S2"])[:12]

ave = tt
test = np.array(dat["volume"])[132:144]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean(ave ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)

# SVR

In [31]:
dat = pd.read_csv('GER19M.csv')
y = np.array(dat["TT"])[:127]
M1 = np.array(dat["M1"])[0:127]
M2 = np.array(dat["M2"])[0:127]
M3 = np.array(dat["M3"])[0:127]
M4 = np.array(dat["M4"])[0:127]
M5 = np.array(dat["M5"])[0:127]

x=np.column_stack((M1,M2,M3,M4,M5))
y.resize(127, 1)

M1t = np.array(dat["M1"])[127:139]
M2t = np.array(dat["M2"])[127:139]
M3t = np.array(dat["M3"])[127:139]
M4t = np.array(dat["M4"])[127:139]
M5t = np.array(dat["M5"])[127:139]

xt=np.column_stack((M1t,M2t,M3t,M4t,M5t))

scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler1.fit(x)
scaler2.fit(y)
x = scaler1.transform(x)  
y = scaler2.transform(y) 
xt = scaler1.transform(xt)

In [32]:
clf=SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
 kernel='rbf', max_iter=500, shrinking=True, tol=0.001, verbose=False)
clf.fit(x, y)

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=500, shrinking=True, tol=0.001, verbose=False)

In [33]:
ave = clf.predict(xt)
print(ave)
ave = scaler2.inverse_transform(ave)
print(ave)

np.savetxt('output.csv', ave, delimiter = ',')

dat = pd.read_csv('GER19.csv')
OO = np.array(dat["S2"])[:12]
test = np.array(dat["volume"])[132:144]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean((ave+OO) ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)

[-0.54466742 -0.53378661  2.2304677   2.46481984 -0.87377266 -1.27587948
  0.0449318  -1.31825835 -0.39516363  0.77126835  1.79326935 -0.3363623 ]
[16197.19851978 16242.19101678 27672.47920911 28641.53352193
 14836.33684771 13173.61106503 18635.21187564 12998.37298918
 16815.40190278 21638.63891707 25864.64874528 17058.54744144]
AE= -999.5976709765597 MAE= 2477.004001303854 RMSE= 3441.358309899487 MAPE= 14.884621014479343 TIC= 0.08862548476006638


In [34]:
dat = pd.read_csv('GERA.csv')

ave = np.array(dat["AVE12"])[:36]
test = np.array(dat["TEST"])[:36]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean(ave ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)

AE= -330.2530140833331 MAE= 2000.4840286944445 RMSE= 2699.5990381555457 MAPE= 11.996424948317003 TIC= 0.06949640674145635


# BPNN

In [39]:
dat = pd.read_csv('GER19M.csv')
y = np.array(dat["TT"])[:127]
M1 = np.array(dat["M1"])[0:127]
M2 = np.array(dat["M2"])[0:127]
M3 = np.array(dat["M3"])[0:127]
M4 = np.array(dat["M4"])[0:127]
M5 = np.array(dat["M5"])[0:127]

x=np.column_stack((M1,M2,M3,M4,M5))
y.resize(127, 1)

M1t = np.array(dat["M1"])[127:139]
M2t = np.array(dat["M2"])[127:139]
M3t = np.array(dat["M3"])[127:139]
M4t = np.array(dat["M4"])[127:139]
M5t = np.array(dat["M5"])[127:139]

xt=np.column_stack((M1t,M2t,M3t,M4t,M5t))

scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler1.fit(x)
scaler2.fit(y)
x = scaler1.transform(x)  
y = scaler2.transform(y) 
xt = scaler1.transform(xt)

In [40]:
output = [ ]

for ss in range(1,101):
   
    reg= MLPRegressor(hidden_layer_sizes=(10, ),max_iter=500)
    reg.fit(x, y)
    
    forecasts = scaler2.inverse_transform(reg.predict(xt))
    output.append(forecasts)
    
ave = []
for j in range(0,12):
    sum = 0
    for i in range(0,100):
        sum += output[i][j]
    ave.append(sum/100)
    
np.savetxt('output.csv', ave, delimiter = ',')
print(ave)

dat = pd.read_csv('GER19.csv')
OO = np.array(dat["S2"])[:12]
test = np.array(dat["volume"])[132:144]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean((ave+OO) ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)
    

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached 

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

[14420.326961858671, 16106.461960715433, 21485.536623674794, 24012.44142050426, 17748.96143749365, 13631.472725010137, 12750.75122615739, 19455.444366108426, 18908.421615870713, 22462.849134782075, 24866.397549069763, 21425.594750164553]
AE= -791.3049809508221 MAE= 3923.628942251636 RMSE= 4818.286147771857 MAPE= 22.275355084176375 TIC= 0.12578026381587218


C:\Users\73972\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [41]:
dat = pd.read_csv('GERA.csv')

ave = np.array(dat["AVE13"])[:36]
test = np.array(dat["TEST"])[:36]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean(ave ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)

AE= -91.53131916666653 MAE= 2692.3109741666667 RMSE= 3436.222284564174 MAPE= 14.919362399243807 TIC= 0.08962438680439411


# ELM

In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
from hpelm import ELM
import math
import pylab as pb
from IPython.display import display
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [55]:
dat = pd.read_csv('ENG19M.csv')
y = np.array(dat["TT"])[:127]
M1 = np.array(dat["M1"])[0:127]
M2 = np.array(dat["M2"])[0:127]
M3 = np.array(dat["M3"])[0:127]
M4 = np.array(dat["M4"])[0:127]
M5 = np.array(dat["M5"])[0:127]

x=np.column_stack((M1,M2,M3,M4,M5))
y.resize(127, 1)

M1t = np.array(dat["M1"])[127:139]
M2t = np.array(dat["M2"])[127:139]
M3t = np.array(dat["M3"])[127:139]
M4t = np.array(dat["M4"])[127:139]
M5t = np.array(dat["M5"])[127:139]

xt=np.column_stack((M1t,M2t,M3t,M4t,M5t))

scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler1.fit(x)
scaler2.fit(y)
x = scaler1.transform(x)  
y = scaler2.transform(y) 
xt = scaler1.transform(xt)

In [56]:
output = [ ]
for ss in range(1,101):
   
    elm = ELM(5, 1)
    elm.add_neurons(10, "tanh")
    elm.train(x, y, "r")
    forecasts = scaler2.inverse_transform(elm.predict(xt))
    
    output.append(forecasts)
    
ave = []
for j in range(0,12):
    sum = 0
    for i in range(0,100):
        sum += output[i][j]
    ave.append(sum/100)
np.savetxt('output.csv', ave, delimiter = ',')
print(ave)

dat = pd.read_csv('ENG19.csv')
OO = np.array(dat["S2"])[:12]
test = np.array(dat["volume"])[132:144]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean((ave+OO) ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)

[array([40624.93851273]), array([41972.6619454]), array([48215.15794646]), array([49800.19432938]), array([43977.01761086]), array([37206.08108745]), array([38994.89884552]), array([46297.70926104]), array([41136.31638557]), array([45041.98531781]), array([51818.89457254]), array([46662.9449586])]
AE= -1114.233397778921 MAE= 8909.7671075189 RMSE= 11040.607374873094 MAPE= 20.957113244026484 TIC= 0.12421162595110312


In [64]:
dat = pd.read_csv('ENGA.csv')

ave = np.array(dat["AVE15"])[:36]
test = np.array(dat["TEST"])[:36]

ae = np.nanmean(test - ave)
rmse = np.sqrt(np.nanmean((test - ave) ** 2))
mae = np.nanmean(np.abs(test - ave))
mape = np.nanmean(np.abs(test - ave)/test)*100
tic = rmse/(np.sqrt(np.nanmean(test ** 2))+np.sqrt(np.nanmean(ave ** 2)))
print('AE=',ae,'MAE=',mae,'RMSE=',rmse,'MAPE=',mape,'TIC=',tic)

AE= -1229.555007777777 MAE= 8727.174544444446 RMSE= 10105.310934205778 MAPE= 19.604240242350713 TIC= 0.10778469183725851
